In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                   # 이미지를 쓰기 금지로 설정
    results = model.process(image)                  # 랜드마크 예측
    image.flags.writeable = True                    # 이미지를 다시 쓰기 가능하도록 설정
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)


def draw_styled_landmarks(image, results):
    # 얼굴 랜드마크 연결선을 특정 스타일로 그림
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
                              )

    # 자세(포즈) 랜드마크 연결선을 특정 스타일로 그림
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
                              )

    # 왼손 랜드마크 연결선을 특정 스타일로 그림
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
                              )

    # 오른손 랜드마크 연결선을 특정 스타일로 그림
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                              )



In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


In [10]:
'''
for i in range(4):  # 예를 들어, 처음 4개의 카메라를 확인
    cap = cv2.VideoCapture(i)
    if not cap.isOpened():
        print(f"카메라 {i}를 찾을 수 없습니다.")
    else:
        print(f"카메라 {i}를 사용합니다.")
        break
'''
import numpy as np
# 웹캠을 초기화
cap = cv2.VideoCapture(0)
# mediapipe 모델을 설정
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        
        # 영상 읽기
        ret, frame = cap.read()

        # 랜드마크 예측
        image, results = mediapipe_detection(frame, holistic)  

        cv2.putText(image, str(extract_keypoints(results)[53]), (40, 40), cv2.FONT_ITALIC, 1, (255, 0, 0), 2)
        cv2.putText(image, str(extract_keypoints(results)[61]), (40, 64), cv2.FONT_ITALIC, 1, (255, 0, 0), 2)

        if extract_keypoints(results)[61] < extract_keypoints(results)[53] or extract_keypoints(results)[65] < extract_keypoints(results)[57]:
            print("Start")

        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # q 키를 누르면 graceful하게 종료
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()


Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start
Start


In [6]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_excel("need_data.xlsx")
# actions = np.array(['싫어하다', '나', '짜증', '너', '사감', '우유', '무섭다', '청소당번', '가다', '지름길', '배고프다', '대박', '답답', '나이', '기분', '못생기다', '때문', '비율', '좋다', '배부르다', '친구', '똑똑하다', '밥', '빠르다', '덥다', '동대문', '춥다', '형편없다', '차다', '온도', '불복종'])
for action in ["나", "동대문", "형편없다", "차다"]:
    dataset = data[data["name"]==action]
    print(dataset.values[-2:])

[['NIA_SL_WORD1157_REAL15_F.mp4' '나' 1.51 2.619 3.634]
 ['NIA_SL_WORD1157_REAL16_F.mp4' '나' 1.006 2.08 3.267]]
[['NIA_SL_WORD1431_REAL15_F.mp4' '동대문' 1.12 2.176 3.4]
 ['NIA_SL_WORD1431_REAL16_F.mp4' '동대문' 1.688 2.857 3.767]]
[['NIA_SL_WORD2384_REAL15_F.mp4' '형편없다' 1.561 2.29 3.334]
 ['NIA_SL_WORD2384_REAL16_F.mp4' '형편없다' 1.027 1.978 2.767]]
[['NIA_SL_WORD2570_REAL15_F.mp4' '차다' 1.523 2.618 3.8]
 ['NIA_SL_WORD2570_REAL16_F.mp4' '차다' 1.568 2.448 3.8]]


In [18]:
data["name"]=="나"

0      False
1      False
2      False
3      False
4      False
       ...  
491    False
492    False
493    False
494    False
495    False
Name: name, Length: 496, dtype: bool

In [18]:
import cv2
import time
import os
cap = cv2.VideoCapture(os.path.join("D:", "/16", "NIA_SL_WORD0953_REAL16_F.mp4"))
start = 1.217*2.474
end = 2.082*2.474

start_time = time.time()
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while (cap.isOpened()):
        ret, img = cap.read()
        if img is None:
            break
        img = cv2.resize(img, (600, 600))
        img, results = mediapipe_detection(img, holistic)
        draw_styled_landmarks(img, results)
        data = extract_keypoints(results)
        
        tiime = time.time() - start_time
        if end > tiime > start:
            print("record")
    
        cv2.imshow('frame', img)
        if cv2.waitKey(1) == ord("q"):
            break
print(time.time()-start_time)
cap.release()
cv2.destroyAllWindows()

record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
record
8.262789964675903
